## Edinburgh Living Landscape Pollinator Pledge

https://edinburghlivinglandscape.org.uk/pollinatorpledge/

In [16]:
import logging
import sys

import folium
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from shapely.geometry import Point

%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
fn = '../data/raw/map-point-submission-do-not-rename-2019-02-15.csv'
pollinator_data = pd.read_csv(fn, parse_dates=['Entry Date'])
pollinator_data = pollinator_data.rename(columns={'What type of space do you have?': 'type'})

In [18]:
df = pollinator_data#[['latitude', 'longitude', 'type']]
df = df.dropna()
df.head()

,Enter your postcode,latitude,longitude,type,Plant for pollinators,Make space for nature,Expand the network,What is your first step going to be?,Entry Id,Entry Date
9,EH29 9LB,55.959762,-3.399653,Small garden,Plant for pollinators,Make space for nature,Expand the network,Put up bee Hotel and plant more widely to hope...,737,2019-06-02 19:02:00
14,Eh10 4na,55.937844,-3.209711,Communal greenspace,Plant for pollinators,Make space for nature,Expand the network,Plant flower seeds this spring,730,2019-06-02 14:59:00
15,EH125TS,55.939713,-3.251081,Small garden,Plant for pollinators,Make space for nature,Expand the network,Add more flowering plants,729,2019-06-02 13:10:00
18,Eh8 7lq,55.947564,-3.142593,Large garden,Plant for pollinators,Make space for nature,Expand the network,Share a post,723,2019-05-02 19:10:00
19,eh8 8hg,55.946015,-3.159635,Communal greenspace,Plant for pollinators,Make space for nature,Expand the network,Plant wild flowers,721,2019-05-02 10:59:00


In [19]:
# locations = list(zip(pollinator_df['latitude'], pollinator_df['longitude']))
# locations[:10]

points = [Point(x, y) for x, y in zip(df.longitude, df.latitude)]

In [20]:
gdf = gpd.GeoDataFrame(df, geometry=points)
gdf.head()

,Enter your postcode,latitude,longitude,type,Plant for pollinators,Make space for nature,Expand the network,What is your first step going to be?,Entry Id,Entry Date,geometry
9,EH29 9LB,55.959762,-3.399653,Small garden,Plant for pollinators,Make space for nature,Expand the network,Put up bee Hotel and plant more widely to hope...,737,2019-06-02 19:02:00,POINT (-3.3996525 55.9597619)
14,Eh10 4na,55.937844,-3.209711,Communal greenspace,Plant for pollinators,Make space for nature,Expand the network,Plant flower seeds this spring,730,2019-06-02 14:59:00,POINT (-3.2097109 55.9378437)
15,EH125TS,55.939713,-3.251081,Small garden,Plant for pollinators,Make space for nature,Expand the network,Add more flowering plants,729,2019-06-02 13:10:00,POINT (-3.251081 55.93971329999999)
18,Eh8 7lq,55.947564,-3.142593,Large garden,Plant for pollinators,Make space for nature,Expand the network,Share a post,723,2019-05-02 19:10:00,POINT (-3.1425928 55.9475635)
19,eh8 8hg,55.946015,-3.159635,Communal greenspace,Plant for pollinators,Make space for nature,Expand the network,Plant wild flowers,721,2019-05-02 10:59:00,POINT (-3.1596354 55.94601529999999)


In [21]:
tiles = "openstreetmap"
edinburgh_centre = (55.953251, -3.188267)

m = folium.Map(location=edinburgh_centre, tiles=tiles, zoom_start=12)

for location in locations:
    folium.CircleMarker(location=location, 
                        radius=5,
                        stroke=1,
                        color='darkblue',
                        fill_color='darkblue').add_to(m)
m

NameError: name 'locations' is not defined